In [22]:
import json

In [23]:
def style(line):
    classes = line.split(".")
    class_dict ={}
    for c in classes:
        c = c.replace("}","")
        s = c.split("{")
        c_id = s[0]
        if len(s)>1:
            if c_id not in class_dict:
                class_dict[c_id]={}
            a = str(s[1])
            formats = a.split(";")
            for f in formats:
                if len(f)>2:
                    fsplit = f.split(":")
                    if "<" not in fsplit[0]:
                        if len(fsplit)>1:
                            class_dict[c_id][fsplit[0]]=fsplit[1]
        else:
            if class_dict:
                s = c.split(",")[0]
                if s in class_dict:
                    del class_dict[s]
    return(class_dict)
            

In [24]:
def extract(svgtype,lines,i,style_dict):
    i+=1
    attributes = {}
    flag = True;
    while "/>" not in lines[i]:
        line = lines[i].replace('"','')
        actions = line.strip().split("=")
        if actions[0]=="class":
            class_id = actions[1]
            if class_id in style_dict:
                attributes = style_dict[class_id].copy()
            else:
                flag = False
        else:
            if flag:
                attributes[actions[0]]=actions[1]
        i+=1
    svg_element = {"svgtype":svgtype,"attributes":attributes}
    return (i,svg_element)

In [25]:
def parse(file_name):
    file_in = open(file_name+'.svg','r')
    file_out = open(file_name+'.json','w')    


    lines = file_in.readlines() 
    style_dict = {}
    strokes=[]
    flag = False
    # Strips the newline character 
    count = 0
    for i in range(len(lines)): 
        if "<style" in lines[i]:
            i+=1
            style_dict = style(lines[i].strip())
            flag = True
        if ("<" in lines[i]) and (flag):
            svg_type = lines[i].strip().replace("<","")
            if ">" not in svg_type:
                i, svg_elemnet = extract(svg_type,lines,i,style_dict)
                svg_elemnet["count"]= count
                strokes.append(svg_elemnet)
                #print(svg_elemnet)
                count+=1
    s = json.dumps({file_name:strokes})
    file_out.write(s)
    file_in.close()
    file_out.close()

In [27]:
#/home/ric/Projects/Comic/Sandbox/img/torso_arms_crossed-01.svg
parse('torso_colour_test-02')
parse('head_colour_test-03')
